In [1]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics

from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

In [2]:
df=pd.read_csv('reddit_preprocessing.csv').dropna(subset=['clean_comment'])
df.shape

(36662, 2)

In [3]:
!pip install mlflow
!pip install pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 72.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 71.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.7/242.7 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 733.8/733.8 kB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 14.3 MB/s eta 0:00:00


In [9]:
from pyngrok import ngrok

# Kill all active tunnels
ngrok.kill()


In [10]:
import mlflow
import mlflow.sklearn

mlflow.set_experiment("Exp 3- Tfidf Trigram max_feature")

from google.colab import userdata
ngRoktoken=userdata.get('ngroktoen')

get_ipython().system_raw('mlflow ui --port 2000 &')
mlflow.set_tracking_uri('http://localhost:2000')
from pyngrok import ngrok
ngrok.set_auth_token(ngRoktoken)

public_url=ngrok.connect(2000).public_url
print('mlflow ui url: ', public_url)

mlflow ui url:  https://a61d-35-245-145-205.ngrok-free.app


In [12]:
# Step 1: Function to run the experiment
def run_experiment_tfidf_max_features(max_features):
    ngram_range = (1, 3)  # Trigrams setting

    # Step 2: Vectorization using TF-IDF with varying max_features
    vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)

    X = vectorizer.fit_transform(df['clean_comment'])
    y = df['category']

    # Step 3: Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

    # Step 4: Define and train a Random Forest model
    with mlflow.start_run() as run:
        # Set tags for the experiment
        mlflow.set_tag("mlflow.runName", f"TFIDF_Trigrams_max_features_{max_features}")
        mlflow.set_tag("experiment_type", "feature_engineering")
        mlflow.set_tag("model_type", "RandomForestClassifier")

        # Add description
        mlflow.set_tag("description", f"Random Forest with TF-IDF Trigrams, max_features={max_features}")

        # Log vectorizer parameters
        mlflow.log_param("vectorizer_type", "TF-IDF")
        mlflow.log_param("ngram_range", ngram_range)
        mlflow.log_param("vectorizer_max_features", max_features)

        # Log Random Forest parameters
        n_estimators = 200
        max_depth = 15

        mlflow.log_param("n_estimators", n_estimators)
        mlflow.log_param("max_depth", max_depth)

        # Initialize and train the model
        model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
        model.fit(X_train, y_train)

        # Step 5: Make predictions and log metrics
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log confusion matrix
        conf_matrix = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(8, 6))
        sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title(f"Confusion Matrix: TF-IDF, max_features={max_features}")
        plt.savefig("confusion_matrix.png")
        mlflow.log_artifact("confusion_matrix.png")
        plt.close()

        # Log the model (✅ updated to remove warnings)
        from mlflow.models.signature import infer_signature

        input_example = X_test[:1]
        pred_example = model.predict(input_example)
        signature = infer_signature(input_example, pred_example)

        mlflow.sklearn.log_model(
            sk_model=model,
            artifact_path=f"random_forest_model_tfidf_{max_features}",
            input_example=input_example,
            signature=signature
        )

# Step 6: Test various max_features values
max_features_values = [1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000]

for max_features in max_features_values:
    run_experiment_tfidf_max_features(max_features)


2025/06/28 06:57:52 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run TFIDF_Trigrams_max_features_1000 at: http://localhost:2000/#/experiments/393214068734621470/runs/38b28bdadda24626877991163edfc250
🧪 View experiment at: http://localhost:2000/#/experiments/393214068734621470


2025/06/28 06:58:11 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run TFIDF_Trigrams_max_features_2000 at: http://localhost:2000/#/experiments/393214068734621470/runs/deb4849bf75746d6ad101492cdeb9048
🧪 View experiment at: http://localhost:2000/#/experiments/393214068734621470


2025/06/28 06:58:29 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


...

🏃 View run TFIDF_Trigrams_max_features_8000 at: http://localhost:2000/#/experiments/393214068734621470/runs/1dc7048fca684052b959a6e31c55c8f8
🧪 View experiment at: http://localhost:2000/#/experiments/393214068734621470


2025/06/28 07:00:27 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run TFIDF_Trigrams_max_features_9000 at: http://localhost:2000/#/experiments/393214068734621470/runs/4967a73b90ee4fbb99dc0dad192b3e52
🧪 View experiment at: http://localhost:2000/#/experiments/393214068734621470


2025/06/28 07:00:47 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run TFIDF_Trigrams_max_features_10000 at: http://localhost:2000/#/experiments/393214068734621470/runs/da19f1fdc4474df496d5042db5de2763
🧪 View experiment at: http://localhost:2000/#/experiments/393214068734621470


In [ ]:
# Step 1: Function to run the experiment
def run_experiment_tfidf_max_features(max_features):
    ngram_range = (1, 3)  # Trigrams setting

    # Step 2: Vectorization using TF-IDF with varying max_features
    vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)

    X = vectorizer.fit_transform(df['clean_comment'])
    y = df['category']

    # Step 3: Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

    # Step 4: Define and train a Random Forest model
    n_estimators = 200
    max_depth = 15

    # Initialize and train the model
    model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
    model.fit(X_train, y_train)

    # Step 5: Make predictions and log metrics
    y_pred = model.predict(X_test)

    # Accuracy
    accuracy = accuracy_score(y_test, y_pred)
    print(f"max_features={max_features} | Accuracy: {accuracy:.4f}")

    # Classification report
    classification_rep = classification_report(y_test, y_pred, output_dict=True)
    for label, metrics in classification_rep.items():
        if isinstance(metrics, dict):
            for metric, value in metrics.items():
                print(f"{label}_{metric}: {value:.4f}")

    # Confusion matrix
    conf_matrix = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.title(f"Confusion Matrix: TF-IDF, max_features={max_features}")
    plt.savefig("confusion_matrix.png")
    plt.close()

# Step 6: Test various max_features values
max_features_values = [1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000]

for max_features in max_features_values:
    run_experiment_tfidf_max_features(max_features)

max_features=1000 | Accuracy: 0.6623
-1_precision: 0.8785
-1_recall: 0.1315
-1_f1-score: 0.2288
-1_support: 1650.0000
0_precision: 0.6307
0_recall: 0.8818
0_f1-score: 0.7354
0_support: 2529.0000
1_precision: 0.6789
1_recall: 0.7641
1_f1-score: 0.7190
1_support: 3154.0000
macro avg_precision: 0.7294
macro avg_recall: 0.5925
macro avg_f1-score: 0.5610
macro avg_support: 7333.0000
weighted avg_precision: 0.7072
weighted avg_recall: 0.6623
weighted avg_f1-score: 0.6143
weighted avg_support: 7333.0000
max_features=2000 | Accuracy: 0.6596
-1_precision: 0.8902
-1_recall: 0.0933
-1_f1-score: 0.1690
-1_support: 1650.0000
0_precision: 0.6446
0_recall: 0.8679
0_f1-score: 0.7398
0_support: 2529.0000
1_precision: 0.6626
1_recall: 0.7888
1_f1-score: 0.7202
1_support: 3154.0000
macro avg_precision: 0.7325
macro avg_recall: 0.5834
macro avg_f1-score: 0.5430
macro avg_support: 7333.0000
weighted avg_precision: 0.7076
weighted avg_recall: 0.6596
weighted avg_f1-score: 0.6029
weighted avg_support: 7333.0